## Load libraries

In [1]:
import pandas as pd
import os
import sys
sys.path.append("../libs")
sys.path.append("../")
from definitions import ROOT_DIR
import utils as utils

# ETL

## 1. Load dataset

In [2]:
# Load dataset from data_files/internet.xlsx - sheet: 'Ingresos '
df = utils.get_xls_sheet_data('/data_files/internet.xlsx', 'Ingresos ')
df.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,2,4.420322e+08,Abr-Jun 2024
1,2024,1,3.461990e+08,Ene-Mar 2024
2,2023,4,1.673760e+08,Oct-Dic 2023
3,2023,3,1.331066e+08,Jul-Sept 2023
4,2023,2,1.180603e+08,Jun-Mar 2023


## 2. Handling missing values

### Look for missing values in all cells

In [3]:
#Find missing values
df.isnull().sum()

Año                          0
Trimestre                    0
Ingresos (miles de pesos)    0
Periodo                      0
dtype: int64

#### There's no missing values

## 3. Look for Duplicates

### Find duplicates for complete rows

In [4]:
#Find duplicates by complete row
df.duplicated().sum()

0

#### There's no complete duplicated rows

### Fin duplicated rows for year and quarter ('Trimestre')

In [5]:
#Find duplicated rows by row, for year and quarter
df.duplicated(subset=['Año', 'Trimestre']).sum()

0

#### There's no duplicated rows for year and quarter

## 4. Finding outliers

In [6]:
#Finding outliers
df.describe()

,Año,Trimestre,Ingresos (miles de pesos)
count,42.000000,42.000000,4.200000e+01
mean,2019.000000,2.452381,5.001648e+07
std,3.728597,1.130560,8.710208e+07
min,2014.000000,1.000000,2.984054e+06
25%,2016.000000,1.250000,7.055326e+06
50%,2019.000000,2.000000,2.047527e+07
75%,2021.000000,3.000000,4.485090e+07
max,2033.000000,4.000000,4.420322e+08


#### In the statistics, we can see that there are outliers values for the column year ("Año"), because the maximum value is 2033 and the data is historical. Now we will review the year column in detail.

In [7]:
df.groupby(['Año','Trimestre']).sum()

Ingresos (miles de pesos)        Periodo
Año  Trimestre                                          
2014 1                       2.984054e+06   Ene-Mar 2014
     2                       3.270816e+06   Abr-Jun 2014
     3                       3.478638e+06  Jul-Sept 2014
     4                       3.950441e+06   Oct-Dic 2014
2015 1                       4.876385e+06   Ene-Mar 2015
     2                       4.701791e+06   Abr-Jun 2015
     3                       5.153739e+06  Jul-Sept 2015
     4                       5.376899e+06   Oct-Dic 2015
2016 1                       5.936845e+06   Ene-Mar 2016
     2                       6.534241e+06   Abr-Jun 2016
     3                       6.912442e+06  Jul-Sept 2016
     4                       7.483980e+06   Oct-Dic 2016
2017 1                       8.701201e+06   Ene-Mar 2017
     2                       9.678647e+06   Abr-Jun 2017
     3                       1.006600e+07  Jul-Sept 2017
     4                       1.122613e+07   Oct-Dic 2017
2018 1                       1.317146e+07   Ene-Mar 2018
     2                       1.431947e+07   Abr-Jun 2018
     3                       1.541098e+07  Jul-Sept 2018
     4                       1.663769e+07   Oct-Dic 2018
2019 1                       2.020741e+07   Ene-Mar 2019
     2                       2.074312e+07   Abr-Jun 2019
     3                       2.165275e+07  Jul-Sept 2019
     4                       2.416925e+07   Oct-Dic 2019
2020 1                       2.994622e+07   Ene-Mar 2020
     2                       3.210248e+07   Abr-Jun 2020
     3                       3.199744e+07  Jul-Sept 2020
     4                       3.353970e+07   Oct-Dic 2020
2021 1                       3.667637e+07   Ene-Mar 2021
     2                       3.823967e+07   Abr-Jun 2021
     3                       4.299994e+07  Jul-Sept 2021
     4                       4.546789e+07   Oct-Dic 2021
2022 1                       5.521931e+07   Ene-Mar 2022
     2                       6.041950e+07   Abr-Jun 2022
     3                       6.408678e+07  Jul-Sept 2022
     4                       7.244348e+07   Oct-Dic 2022
2023 2                       1.180603e+08   Jun-Mar 2023
     3                       1.331066e+08  Jul-Sept 2023
     4                       1.673760e+08   Oct-Dic 2023
2024 1                       3.461990e+08   Ene-Mar 2024
     2                       4.420322e+08   Abr-Jun 2024
2033 1                       1.041350e+08   Ene-Mar 2023

#### We can see there's no 1st quarter data for 20**2**3 and we have 1st quarter data for 20**3**3. I think there is a typing error and the correct year should be 2023. And in the column "Periodo" we have "End-Mar 2023", this confirms the hypothesis.  We will correct this error.

In [8]:
#Change year 2033 to 2023
df['Año'] = df['Año'].replace(2033, 2023)
df.groupby(['Año','Trimestre']).sum()

Ingresos (miles de pesos)        Periodo
Año  Trimestre                                          
2014 1                       2.984054e+06   Ene-Mar 2014
     2                       3.270816e+06   Abr-Jun 2014
     3                       3.478638e+06  Jul-Sept 2014
     4                       3.950441e+06   Oct-Dic 2014
2015 1                       4.876385e+06   Ene-Mar 2015
     2                       4.701791e+06   Abr-Jun 2015
     3                       5.153739e+06  Jul-Sept 2015
     4                       5.376899e+06   Oct-Dic 2015
2016 1                       5.936845e+06   Ene-Mar 2016
     2                       6.534241e+06   Abr-Jun 2016
     3                       6.912442e+06  Jul-Sept 2016
     4                       7.483980e+06   Oct-Dic 2016
2017 1                       8.701201e+06   Ene-Mar 2017
     2                       9.678647e+06   Abr-Jun 2017
     3                       1.006600e+07  Jul-Sept 2017
     4                       1.122613e+07   Oct-Dic 2017
2018 1                       1.317146e+07   Ene-Mar 2018
     2                       1.431947e+07   Abr-Jun 2018
     3                       1.541098e+07  Jul-Sept 2018
     4                       1.663769e+07   Oct-Dic 2018
2019 1                       2.020741e+07   Ene-Mar 2019
     2                       2.074312e+07   Abr-Jun 2019
     3                       2.165275e+07  Jul-Sept 2019
     4                       2.416925e+07   Oct-Dic 2019
2020 1                       2.994622e+07   Ene-Mar 2020
     2                       3.210248e+07   Abr-Jun 2020
     3                       3.199744e+07  Jul-Sept 2020
     4                       3.353970e+07   Oct-Dic 2020
2021 1                       3.667637e+07   Ene-Mar 2021
     2                       3.823967e+07   Abr-Jun 2021
     3                       4.299994e+07  Jul-Sept 2021
     4                       4.546789e+07   Oct-Dic 2021
2022 1                       5.521931e+07   Ene-Mar 2022
     2                       6.041950e+07   Abr-Jun 2022
     3                       6.408678e+07  Jul-Sept 2022
     4                       7.244348e+07   Oct-Dic 2022
2023 1                       1.041350e+08   Ene-Mar 2023
     2                       1.180603e+08   Jun-Mar 2023
     3                       1.331066e+08  Jul-Sept 2023
     4                       1.673760e+08   Oct-Dic 2023
2024 1                       3.461990e+08   Ene-Mar 2024
     2                       4.420322e+08   Abr-Jun 2024

## 5. Data Types

### We will review the data types for each column.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        42 non-null     int64  
 1   Trimestre                  42 non-null     int64  
 2   Ingresos (miles de pesos)  42 non-null     float64
 3   Periodo                    42 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ KB


#### The data types are consistent with the data provided. Just the column "Periodo" have an object data type, and have strings values. We should change the data type to string, but since the data is redundant with the quarter ("trimestre") column, we will drop it.

In [10]:
#drop Periodo column
df = df.drop('Periodo', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        42 non-null     int64  
 1   Trimestre                  42 non-null     int64  
 2   Ingresos (miles de pesos)  42 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.1 KB


# EDA